In [1]:
!pip install transformers accelerate sentence-transformers evaluate nltk textstat --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 140.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 470.2/470.2 kB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.1/239.1 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 558.8/558.8 kB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 124.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 97.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import evaluate
import textstat
import nltk
import numpy as np
import torch

nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
from huggingface_hub import login
login()

In [5]:
model_name = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")


/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:999: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [7]:
from google.colab import drive
drive.mount('/content/drive')

test_df = pd.read_excel("/content/drive/My Drive/Colab Notebooks/DATASCI 266/Final Project/test_set.xlsx")
test_df_sample = test_df.sample(n=100, random_state=42)

PROMPT_VARIANTS = {
    "P1": "Given this rap line, generate the next line: {line1}"
}


Mounted at /content/drive


In [11]:
!pip install rouge_score
!pip install bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.0 MB/s eta 0:00:00


In [12]:
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")
bertscore = evaluate.load("bertscore")
sbert = SentenceTransformer("all-MiniLM-L6-v2")

def calc_bleu(preds, refs):
    return bleu.compute(predictions=preds, references=refs)["bleu"]

def calc_rouge(preds, refs):
    r = rouge.compute(predictions=preds, references=refs)
    return r["rouge1"], r["rouge2"], r["rougeL"]

def calc_bertscore(preds, refs):
    b = bertscore.compute(predictions=preds, references=refs, lang="en")
    return np.mean(b["f1"])

def calc_sbert(preds, refs):
    sims = [util.cos_sim(sbert.encode(p), sbert.encode(r))[0][0].item() for p, r in zip(preds, refs)]
    return np.mean(sims)

def rhyme_score(gen, ref):
    g_last = gen.strip().split()[-1][-2:] if gen.strip() else ""
    r_last = ref.strip().split()[-1][-2:] if ref.strip() else ""
    return int(g_last == r_last)

def syllable_diff(g, r):
    return abs(textstat.syllable_count(g) - textstat.syllable_count(r))

def length_ratio(g, r):
    return len(g.split()) / max(len(r.split()), 1)

def diversity(texts):
    all_words = " ".join(texts).split()
    return len(set(all_words)) / max(len(all_words), 1)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
def evaluate_llama_prompt(prompt_id, prompt_template, model, tokenizer, test_df, max_new_tokens=30):
    prompts, generated, references = [], [], []
    rhyme_scores, syll_diffs, len_ratios = [], [], []

    for _, row in test_df.iterrows():
        line1 = row["line1"]
        line2 = row["line2"]
        prompt = prompt_template.format(line1=line1.strip())

        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                pad_token_id=tokenizer.eos_token_id,
                do_sample=True,
                temperature=0.8,
                top_p=0.95
            )
        gen_line = tokenizer.decode(outputs[0], skip_special_tokens=True).replace(prompt, "").strip()

        prompts.append(prompt)
        generated.append(gen_line)
        references.append(line2)
        rhyme_scores.append(rhyme_score(gen_line, line2))
        syll_diffs.append(syllable_diff(gen_line, line2))
        len_ratios.append(length_ratio(gen_line, line2))

    # Aggregate metrics
    bleu_score = calc_bleu(generated, references)
    r1, r2, rL = calc_rouge(generated, references)
    bert_f1 = calc_bertscore(generated, references)
    sbert_sim = calc_sbert(generated, references)
    div = diversity(generated)

    print(f"\n📊 Prompt: {prompt_id}")
    print(f"BLEU: {bleu_score:.4f}, ROUGE-1: {r1:.4f}, ROUGE-2: {r2:.4f}, ROUGE-L: {rL:.4f}")
    print(f"BERTScore F1: {bert_f1:.4f}, SBERT: {sbert_sim:.4f}, Diversity: {div:.4f}")
    print(f"Avg Rhyme Rate: {np.mean(rhyme_scores):.4f}, Avg Syllable Diff: {np.mean(syll_diffs):.2f}, Length Ratio: {np.mean(len_ratios):.2f}")

    return pd.DataFrame({
        "prompt_id": prompt_id,
        "prompt": prompts,
        "generated_line": generated,
        "reference_line": references,
        "rhyme": rhyme_scores,
        "syllable_diff": syll_diffs,
        "length_ratio": len_ratios,
    })


In [16]:
import pandas as pd
from IPython.display import display

all_results = []
for prompt_id, prompt_template in PROMPT_VARIANTS.items():
    df_result = evaluate_llama_prompt(prompt_id, prompt_template, model, tokenizer, test_df_sample)
    all_results.append(df_result)

final_df = pd.concat(all_results)

# Show result (either head or full)
print(final_df.head(10))  # Show top 10 rows
# OR
from IPython.display import display
display(final_df)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)



📊 Prompt: P1
BLEU: 0.0065, ROUGE-1: 0.0760, ROUGE-2: 0.0127, ROUGE-L: 0.0699
BERTScore F1: 0.8172, SBERT: 0.1792, Diversity: 0.3179
Avg Rhyme Rate: 0.0100, Avg Syllable Diff: 11.52, Length Ratio: 2.41
  prompt_id                                             prompt  \
0        P1  Given this rap line, generate the next line: G...   
1        P1  Given this rap line, generate the next line: W...   
2        P1  Given this rap line, generate the next line: L...   
3        P1  Given this rap line, generate the next line: N...   
4        P1  Given this rap line, generate the next line: B...   
5        P1  Given this rap line, generate the next line: I...   
6        P1  Given this rap line, generate the next line: A...   
7        P1  Given this rap line, generate the next line: S...   
8        P1  Given this rap line, generate the next line: A...   
9        P1  Given this rap line, generate the next line: S...   

                                      generated_line  \
0  "Girl, whate

,prompt_id,prompt,generated_line,reference_line,rhyme,syllable_diff,length_ratio
0,P1,"Given this rap line, generate the next line: G...","""Girl, whatever you want ooh-ooh-ooh"" could po...",Just keep me inside,0,24,4.500000
1,P1,"Given this rap line, generate the next line: W...",(Note: The rhyme scheme and meter of the rap l...,"I'm in the drivers seat, but you can ride along",0,18,2.400000
2,P1,"Given this rap line, generate the next line: L...","roar, like a Harley in a __________ (fill in t...",Let 'em hear them loud pipes,0,15,3.333333
3,P1,"Given this rap line, generate the next line: N...",", I got the ...\n– Nash effect, I got the flow...",She tell me that I got trust issues,0,14,3.000000
4,P1,"Given this rap line, generate the next line: B...",", It keeps on going, like a _____________.\n\n...",A shot callin if I fall then my thoughts gon' ...,0,7,1.727273
...,...,...,...,...,...,...,...
95,P1,"Given this rap line, generate the next line: ""...",Please provide the next line of the rap as a r...,Uh,0,12,11.000000
96,P1,"Given this rap line, generate the next line: D...",", like a magic show.\n\nPlease provide the nex...",Do it like woah,0,10,3.250000
97,P1,"Given this rap line, generate the next line: A...",.\n\nI want the next line to be a play on word...,Might as well go for the gusto now,0,19,3.000000
98,P1,"Given this rap line, generate the next line: P...","Next line:\nFuckin' 'em like I stole it, ain't...","Yeah, and it don't quit",0,12,3.000000
